# 💡 Day 6: Lanternfish
*A massive school of glowing lanternfish swims past.*

## Part 1

### ✨ Building a LanternFish class

In [1]:
from dataclasses import dataclass

@dataclass
class LanternFish:
    """Represents a lanternfish with an internal clock"""
    clock: int = 8
        
    def decrease_clock_count(self):
        """Decreases the clock count by one day"""
        self.clock -= 1
        
    def create_son(self):
        """Resets the clock count and create a new lanternfish"""
        self.clock = 6
        son = LanternFish()
        return son
        
    def update_internal_clock(self, list_of_fishes):
        """Updates the internal clock by decreasing time or creating a new fish"""
        # Not ready to reproduce
        if self.clock > 0:
            self.decrease_clock_count()
            return
        
        # Create a new son and add it to the list of lanternfishes
        son = self.create_son()
        list_of_fishes.append(son)
        

### 🧩 Solving the puzzle

In [14]:
number_of_days = 18

fishes = [LanternFish(clock) for clock in [3, 4, 3, 1, 2]]

for _ in range(number_of_days):
    old_fishes = fishes.copy()
    for lantern in old_fishes:
        lantern.update_internal_clock(fishes)

len(fishes)

5

## Part 2

Seems like we don't really need to add any new features to our code to solve part 2! We only need to change the input number!... But once you try to run it 😧😫... good luck with that. So, what is the problem, and how can we solve it?

Well, as the population grows, we get a TON of fishes to update at each time point, to the point it no longer becomes feasible. **We have to think of ways do decrease the number of computations we use**. My first guess was to avoid keeping track of fishes that share the same initial clock. If we know that a fish starting at clock time 3 will create 5 fishes by day 18, then, instead of following other fishes that share the same initial clock time through the simulation we can follow one and then **multiply the number of offspring by the number of initial fishes.**


In [17]:
number_of_days = 18
initial_time_clock = 3
fishes = [LanternFish(initial_time_clock)]

# Simulate population growth
for _ in range(number_of_days):
    old_fishes = fishes.copy()
    for lantern in old_fishes:
        lantern.update_internal_clock(fishes)

print(f"A fish with time clock {initial_time_clock} produced {len(fishes)} fishes by time {number_of_days}")

A fish with time clock 3 produced 5 fishes by time 18


Thus, we should expect that, by day 10, 2 fishes with this initial time clock will produce 10 new fishes. 

We can **solve our problem for each fish with a unique time clock value** and then use the results to **estimate the number of fishes that our input will produce**. For instance, in our test sample this helps us to avoid repeating the computations for the fishes with a time clock of 3.

In [35]:
number_of_days = 18
unique_clocks = [clock for clock in set([3, 4, 3, 1, 2])]
offspring_dict = {}

fishes = [LanternFish(clock) for clock in unique_clocks]
fish_clocks = [fish.clock for fish in fishes]

# Simulate population growth for each fish
for initial_clock, fish in zip(fish_clocks, fishes):
    simulated_fishes = [fish]
    for _ in range(number_of_days):
        old_simulated_fishes = simulated_fishes.copy()
        for lantern in old_simulated_fishes:
            lantern.update_internal_clock(simulated_fishes)
    # Store the size of the population that the fish produced
    offspring_dict[initial_clock] = len(simulated_fishes)    

offspring_dict

{1: 7, 2: 5, 3: 5, 4: 4}

Knowing how many fishes each initial fish will create, we can sum the number of children for each fish in our input.

In [38]:
# Add the sum of the populations created by each fish
sum_of_fishes = sum([offspring_dict[clock] for clock in [3, 4, 3, 1, 2]])
sum_of_fishes

26

Unfortunately, this is not enough to solve our problem with the input data.

In [9]:
number_of_days = 128

fishes = [LanternFish(clock) for clock in [0, 1, 2, 3, 4, 5, 6, 7, 8]]
counter = {}
fishes_at_half = []

for i, fish in enumerate(fishes):
    simulated_fishes = [fish]
    for _ in range(number_of_days):
        old_simulated_fishes = simulated_fishes.copy()
        for lantern in old_simulated_fishes:
            lantern.update_internal_clock(simulated_fishes)
    counter[i] = len(simulated_fishes)
    fishes_at_half.append([half_fish.clock for half_fish in simulated_fishes])


running fish 1
running fish 2
running fish 3
running fish 4
running fish 5
running fish 6
running fish 7
running fish 8
running fish 9


In [10]:
count = 0
for i in [3, 4, 3 , 1, 2]:
    count += sum([counter[clock] for clock in fishes_at_half[i]])
    
count

26984457539

In [11]:
with open("src/day6/input.txt") as fileobject:
    lines = fileobject.read().splitlines()
    clocks = [int(value) for value in lines[0].split(',')]
    
count = 0
for i in clocks:
    count += sum([counter[clock] for clock in fishes_at_half[i]])
    
count

1572643095893

Welp, this works for this timespan, but doesn't seem to escalate well.